In [1]:
import findspark

In [2]:
findspark.init('/home/samet/spark-3.2.1-bin-hadoop3.2')

In [3]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [16]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website', 'Length of Membership'],
                            outputCol='features')

In [23]:
output = assembler.transform(data)

In [24]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [26]:
final_data = output.select('features', 'Yearly Amount Spent')

In [27]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [28]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [29]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                336|
|   mean| 498.68963576411414|
| stddev|  76.95969603245727|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                164|
|   mean|  500.5933019058113|
| stddev|  84.16281334117038|
|    min|  282.4712457199145|
|    max|  744.2218671047146|
+-------+-------------------+



In [31]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [33]:
lr_model = lr.fit(train_data)

22/04/26 15:10:24 WARN Instrumentation: [1d57ca96] regParam is zero, which might cause numerical instability and overfitting.


In [34]:
test_results = lr_model.evaluate(test_data)

In [35]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-3.4618171675975304|
|-7.2908804084731855|
| 11.611646210029221|
|  3.619932465832278|
| -4.531259705669754|
| -5.121701257354857|
| -7.816907558404807|
|  5.211708787484724|
|  17.95866079975241|
|  7.565104268769744|
| 1.9652367042427272|
|-11.758574637514585|
|-2.1058594324464934|
|  8.049685771406246|
|-1.5413271590633713|
| -5.295942582888927|
| -16.86508532961585|
|-0.9686250978399471|
|  -13.1403537475353|
| -6.615136786968037|
+-------------------+
only showing top 20 rows



/home/samet/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [36]:
test_results.rootMeanSquaredError

9.446501161507133

In [37]:
test_results.r2

0.9873247153965485

In [38]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [39]:
unlabeled_data = test_data.select('features')

In [40]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.7366356860502...|
|[31.8093003166791...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8627411090001...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9120759292006...|
|[31.9365486184489...|
|[31.9453957483445...|
+--------------------+
only showing top 20 rows



In [41]:
predictions = lr_model.transform(unlabeled_data)

In [42]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|285.93306288751205|
|[31.1280900496166...| 564.5435671555279|
|[31.1695067987115...| 415.7448845922636|
|[31.4459724827577...| 481.2570324692963|
|[31.5147378578019...|494.34374770213117|
|[31.5257524169682...|449.08732806723674|
|[31.5261978982398...| 416.9114337507426|
|[31.5316044825729...|431.30389694187784|
|[31.6098395733896...|426.58688885135575|
|[31.6548096756927...|467.69831945877877|
|[31.7366356860502...| 494.9682095512892|
|[31.8093003166791...| 548.5304740003558|
|[31.8186165667690...|448.52453280258214|
|[31.8209982016720...| 416.6255952418071|
|[31.8627411090001...| 557.8394683331101|
|[31.8745516945853...| 397.5811868291564|
|[31.9048571310136...|  490.814942752432|
|[31.9120759292006...|388.50334140354767|
|[31.9365486184489...| 440.3397386428635|
|[31.9453957483445...|   663.63506072462|
+--------------------+------------